In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()
driver.implicitly_wait(10)
driver.set_page_load_timeout(5)

In [2]:
df = pd.read_csv('match_results.csv')
link = df.sample(1)
link2 = link['score_card_link'].iloc[0]
driver.get(link2)

try:
    junk_click_box = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="wzrk-cancel"]'))
    )
    junk_click_box.click()
except NoSuchElementException:
    print('No Such element')

In [8]:
def process_batting_table(xpath):
    batting = driver.find_element(By.XPATH,xpath)
    batting_outer_html = batting.get_attribute('outerHTML')
    batting = pd.read_html(batting_outer_html)[0] 
    batting = (batting
                .dropna(axis = 0, how = 'all')
                .dropna(axis = 1, how = 'all')
                .rename(columns = {'Unnamed: 1':'Dismissal'})
    )
    # last 4 elements are junk and delete the last table
    return batting.iloc[:-4].dropna(axis = 1, how = 'all')

def process_bowling_table(xpath):
    bowling = driver.find_element(By.XPATH,xpath)
    bowling = pd.read_html(bowling.get_attribute('outerHTML'))[0] 
    bowling = bowling[~bowling['BOWLING'].str.contains('\d', regex = True)]
    bowling = (bowling
                .dropna(axis = 0, how = 'all')
                .dropna(axis = 1, how = 'all')
    )

    return bowling


In [9]:
team_one_batting_xpath = "(//table[@class='ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table'])[1]"
team_two_batting_xpath = "(//table[@class='ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table'])[2]"
team_one_bowling_xpath = '(//table[@class="ds-w-full ds-table ds-table-md ds-table-auto "])[1]'
team_two_bowling_xpath = '(//table[@class="ds-w-full ds-table ds-table-md ds-table-auto "])[2]'

In [10]:
# process batting 
batting_one_df = process_batting_table(team_one_batting_xpath)
batting_two_df = process_batting_table(team_two_batting_xpath)
batting_one_df['team_inning'] = link['team_one'].iloc[0]
batting_two_df['team_inning'] = link['team_two'].iloc[0]
batting_one_df['match_id'] = link['score_card'].iloc[0]
batting_two_df['match_id'] = link['score_card'].iloc[0]


In [11]:
# process bowling 
bowling_one_df = process_bowling_table(team_one_bowling_xpath)
bowling_two_df =process_bowling_table(team_two_bowling_xpath)
bowling_one_df['team_inning'] = link['team_one'].iloc[0]
bowling_two_df['team_inning'] = link['team_two'].iloc[0]
bowling_one_df['match_id'] = link['score_card'].iloc[0]
bowling_two_df['match_id'] = link['score_card'].iloc[0]

In [12]:
batting = driver.find_element(By.XPATH,team_one_batting_xpath)

In [17]:
batting_outer_html = batting.get_attribute('outerHTML')
batting_soup = BeautifulSoup(batting_outer_html)
player_links = batting_soup.find_all(class_ = 'ds-inline-flex ds-items-start ds-leading-none') 
player_links = [a['href'] for a in player_links]
player_links

['/player/kl-rahul-422108',
 '/player/rohit-sharma-34102',
 '/player/virat-kohli-253802',
 '/player/suryakumar-yadav-446507',
 '/player/hardik-pandya-625371',
 '/player/rishabh-pant-931581',
 '/player/ravichandran-ashwin-26421',
 '/player/axar-patel-554691',
 '/player/bhuvneshwar-kumar-326016',
 '/player/mohammed-shami-481896',
 '/player/arshdeep-singh-1125976']